# 0. Premilinary Works
How big is the file, number of rows, etc.

In [1]:
cd ../data

/home/angelrps/git/MasterDataScience_FinalProject/data


In [3]:
!ls -Rl
# 2017_Yellow_Taxi_Trip_Data.csv is 10532305815 bytes (9,80 GB)
# 2018_Yellow_Taxi_Trip_Data.csv is 10428263736 bytes (9,71 GB)
# 2019_Yellow_Taxi_Trip_Data.csv is 8197837930 bytes (7,63 GB)

.:
total 28491420
-rwxrwxrwx 1 angelrps angelrps 10532305815 Mar 25 13:15 2017_Yellow_Taxi_Trip_Data.csv
-rwxrwxrwx 1 angelrps angelrps 10428263736 Jun 18 12:58 2018_Yellow_Taxi_Trip_Data.csv
-rwxrwxrwx 1 angelrps angelrps  8197837930 Apr  2 18:02 2019_Yellow_Taxi_Trip_Data.csv
-rwxrwxrwx 1 angelrps angelrps      564780 Jun 18 11:15 Data_Weather_Cleaned.csv
-rwxrwxrwx 1 angelrps angelrps      436485 Jun 16 10:51 LCD_documentation.pdf
-rwxrwxrwx 1 angelrps angelrps    13843633 Jun 16 09:40 NOAA_CentralPark_Weather.csv
-rwxrwxrwx 1 angelrps angelrps      196848 Apr  7 19:34 data_dictionary_trip_records_yellow.pdf
-rwxrwxrwx 1 angelrps angelrps     1489310 Apr  7 19:34 taxi_zone_map_manhattan.jpg
-rwxrwxrwx 1 angelrps angelrps      202694 Apr  4 12:46 trip_record_user_guide.pdf


In [4]:
# I am dealing with very heavy files so I will compress them (bz2) to work with them
# and remove the heavy .csv from my drive
!bzip2 2017_Yellow_Taxi_Trip_Data.csv 2018_Yellow_Taxi_Trip_Data.csv 2019_Yellow_Taxi_Trip_Data.csv

In [5]:
!ls -Rl

.:
total 2897760
-rwxrwxrwx 1 angelrps angelrps 1048588401 Mar 25 13:15 2017_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps 1065918551 Jun 18 12:58 2018_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps  835929368 Apr  2 18:02 2019_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps     564780 Jun 18 11:15 Data_Weather_Cleaned.csv
-rwxrwxrwx 1 angelrps angelrps     436485 Jun 16 10:51 LCD_documentation.pdf
-rwxrwxrwx 1 angelrps angelrps   13843633 Jun 16 09:40 NOAA_CentralPark_Weather.csv
-rwxrwxrwx 1 angelrps angelrps     196848 Apr  7 19:34 data_dictionary_trip_records_yellow.pdf
-rwxrwxrwx 1 angelrps angelrps    1489310 Apr  7 19:34 taxi_zone_map_manhattan.jpg
-rwxrwxrwx 1 angelrps angelrps     202694 Apr  4 12:46 trip_record_user_guide.pdf


In [8]:
# Count Number of Lines
!zgrep -c $ 2017_Yellow_Taxi_Trip_Data.csv.bz2
!zgrep -c $ 2018_Yellow_Taxi_Trip_Data.csv.bz2
!zgrep -c $ 2019_Yellow_Taxi_Trip_Data.csv.bz2

# 2017_...10.748.623 lines
# 2018_...10.734.230 lines
# 2019_...8.402.723 lines

10748623
10734230
8402723


#### I will start exploring the 2019 dataset
I will try to use bash to clean the data as much as I can before importing a Data Frame because I believe is faster.
From a previous exploration I concluded:
1. There are trips with 0 'trip_distance'. I will remove all trips with 'trip_distance' < 0.06 (100 meters) because I consider them either measurement errors or non representative data.
2. There are trips with either negative or very little 'fare_amount' (~0.01). I will remove rows with fare amount<1$ because I consider them measurement errors.

In [11]:
!bzcat ./2019_Yellow_Taxi_Trip_Data.csv.bz2 | head -n 5

# Conclusions:
# 1. There is a header on the first line
# 2. The separator is ','

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
1,04/29/2019 11:31:03 AM,04/29/2019 11:31:03 AM,1,0,1,N,231,264,2,2.5,2.5,0.5,0,0,0.3,5.8,2.5
2,04/29/2019 10:52:08 AM,04/29/2019 11:06:18 AM,1,1.01,1,N,186,230,1,10,0,0.5,2.66,0,0.3,15.96,2.5
2,04/29/2019 11:28:40 AM,04/29/2019 11:33:01 AM,1,0.82,1,N,238,151,1,5,0,0.5,1.66,0,0.3,9.96,2.5
1,04/29/2019 11:28:06 AM,04/29/2019 12:07:32 PM,1,12.1,1,N,138,88,1,38.5,2.5,0.5,10.4,0,0.3,52.2,2.5

bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = ./2019_Yellow_Taxi_Trip_Data.csv.bz2, output file = (stdout)


In [59]:
# LOAD LIBRARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [21]:
import pandas as pd
import random
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
dfsample = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',', nrows=9999,
                      parse_dates={'pickup_datetime':['tpep_pickup_datetime']})
dfsample.shape

(9999, 18)

In [34]:
# I WILL SUMMARIZE ALL STEPS IN EACH CELL

import pandas as pd
import random
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
dfsample = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',', nrows=9999,
                      parse_dates={'pickup_datetime':['tpep_pickup_datetime']})
dfsample.shape

# 2. Remove all trips with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)
dfsample[dfsample['trip_distance']<0.06].index # this should be none

Int64Index([], dtype='int64')

In [52]:
# I WILL SUMMARIZE ALL STEPS IN EACH CELL

import pandas as pd
import random
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
dfsample = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',', nrows=9999,
                      parse_dates={'pickup_datetime':['tpep_pickup_datetime']})
dfsample.shape

# 2. Remove all rows with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)

# 3. Remove rows with 'fare_amount' negative or <1$ because I consider them measurement errors or non relevant.
FareIndex_to_drop = dfsample[dfsample['fare_amount']<0.01].index
dfsample.drop(FareIndex_to_drop, inplace=True)
dfsample[dfsample['fare_amount']<0.01] # this should be none

,pickup_datetime,VendorID,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge


In [57]:
# I WILL SUMMARIZE ALL STEPS IN EACH CELL

import pandas as pd
import random
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
dfsample = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',', nrows=9999,
                      parse_dates={'pickup_datetime':['tpep_pickup_datetime']})
dfsample.shape

# 2. Remove all rows with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)

# 3. Remove rows with 'fare_amount' negative or <1$ because I consider them measurement errors or non relevant.
FareIndex_to_drop = dfsample[dfsample['fare_amount']<0.01].index
dfsample.drop(FareIndex_to_drop, inplace=True)

# 4. Select columns of interest.
#    'pickup_datetime'
#    'PULocationID'
dfsample = dfsample[['pickup_datetime','PULocationID']]
dfsample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9899 entries, 1 to 9998
Data columns (total 2 columns):
pickup_datetime    9899 non-null datetime64[ns]
PULocationID       9899 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 232.0 KB


In [58]:
dfsample.plot()

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f43824fbdd0> (for post_execute):


ValueError: view limit minimum -36854.72785763889 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units